In [ ]:
%run "../library.py"

In [67]:
from sklearn import preprocessing
from sklearn.lda import LDA
from sklearn import svm, cross_validation

In [75]:
# prepare full training and test dataset
# http://stats.stackexchange.com/questions/82923/mixing-continuous-and-binary-data-with-linear-svm
train_data = preprocessing.scale(np.hstack([train_cnn,train_bow,train_attribs]), axis=0)
# train_data = np.hstack([train_data,train_attribs])
test_data = np.hstack([test_cnn,test_bow,test_attribs])
# test_data = np.hstack([test_data,test_attribs])

means = np.mean(train_data,axis=0)
stds = np.std(train_data,axis=0)
test_data = (test_data - means[None,:])/ stds[None,:]

In [53]:
#svm classification
from sklearn import linear_model, svm
from sklearn.cross_validation import KFold

svm_accuracy = []
for train, test in  KFold(len(train_data),n_folds=3):
    X_train,Y_train, X_test,Y_test = train_data[train],train_labels[train],train_data[test],train_labels[test]
    score = svm.SVC(kernel='linear').fit(X_train,Y_train).score(X_test,Y_test)
    svm_accuracy.append(score)
    print score
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

0.374
0.371
0.338
[0.374, 0.371, 0.33800000000000002] mean:  0.361


In [62]:
# LDA
sklearn_lda = LDA(n_components=200)
train_data_lda = sklearn_lda.fit_transform(train_data, train_labels)

/Users/2359media/anaconda/lib/python2.7/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [85]:
svm_accuracy = []
for train, test in  KFold(len(train_data_lda),n_folds=5):
    X_train,Y_train, X_test,Y_test = train_data_lda[train],train_labels[train],train_data_lda[test],train_labels[test]
#     score = svm.SVC(kernel='linear').fit(X_train,Y_train).score(X_test,Y_test)
    predict = svm.SVC(kernel='linear').fit(X_train,Y_train).predict(X_test)
#     kaggle_output(predict)
    print predict
    print Y_test
    
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

['yard' 'hill' 'ranch_house' 'water_tower' 'lab_classroom' 'mine' 'bridge'
 'ramp' 'playground' 'chapel' 'water_mill' 'aquatic_theater' 'armory'
 'fence' 'cliff' 'mountain' 'gorge' 'forest_road' 'tea_garden' 'flood'
 'food_court' 'bus_shelter' 'windmill' 'schoolhouse' 'valley' 'barn'
 'fence' 'town_house' 'orchard' 'sea_cliff' 'nightclub' 'food_court'
 'cheese_factory' 'physics_laboratory' 'desert_road' 'pavilion'
 'soccer_field' 'resort' 'street' 'watering_hole' 'skyscraper' 'hill'
 'ramp' 'kindergarden_classroom' 'dorm_room' 'attic' 'courthouse'
 'conference_room' 'playground' 'ballroom' 'boat_deck' 'snowfield'
 'football_field' 'playground' 'recreation_room' 'nightclub' 'lawn'
 'water_mill' 'campsite' 'building_facade' 'castle' 'water_tower'
 'cottage_garden' 'jewelry_shop' 'lecture_room' 'playground' 'pasture'
 'water_mill' 'river' 'mountain' 'schoolhouse' 'recreation_room'
 'boat_deck' 'gorge' 'water_mill' 'tower' 'bedroom' 'parking_lot' 'freeway'
 'amusement_arcade' 'dining_car' 

In [80]:
#scale test set

test_data_lda = sklearn_lda.transform(test_data)
predict = svm.SVC(kernel='linear').fit(train_data_lda,train_labels).predict(test_data_lda)
kaggle_output(predict)

[['0000.jpg' 'soccer_field']
 ['0001.jpg' 'city']
 ['0002.jpg' 'recreation_room']
 ..., 
 ['0997.jpg' 'fastfood_restaurant']
 ['0998.jpg' 'dorm_room']
 ['0999.jpg' 'pasture']]


In [70]:
# determine accuracy
clustered_names = []
for output in spectral_labels:
    clustered_names.append(spectral_output_names[output])
np.sum(clustered_names == train_labels)*1.0/len(train_labels)

0.27400000000000002

In [30]:
# cluster_labels = 
most_common(cluster_labels)

'closet'

In [75]:
# https://code.google.com/p/scipy-cluster/
# https://www.youtube.com/watch?v=qUQf1JxnTnY
# https://github.com/scikit-fuzzy/scikit-fuzzy

import skfuzzy as fuzz
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(cluster_train[0:500], 200, 2, error=0.005, maxiter=1000, init=None)

In [76]:
d

array([[ 0.60805752,  0.59827038,  0.55244145, ...,  0.3588133 ,
         0.28635455,  0.27066973],
       [ 0.60805856,  0.59827141,  0.55244237, ...,  0.35881338,
         0.28635413,  0.27066924],
       [ 0.60805328,  0.5982662 ,  0.55243768, ...,  0.35881299,
         0.28635625,  0.27067173],
       ..., 
       [ 0.60805807,  0.59827093,  0.55244194, ...,  0.35881334,
         0.28635433,  0.27066947],
       [ 0.60805875,  0.5982716 ,  0.55244254, ...,  0.35881339,
         0.28635405,  0.27066914],
       [ 0.25582793,  0.28317184,  0.32211049, ...,  0.57461935,
         0.61842038,  0.6215155 ]])